# Exploratory Data Analysis for WATER DEMAND

## 1. Load Required Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pylab import rcParams
import sklearn
import warnings
warnings.filterwarnings("ignore")

sns.set(font_scale = 0.7)
plt.rcParams['axes.unicode_minus'] = False

## 2. Load Data

In [ ]:
folder_name = '특광역시'
CITY_NAME_Kor = '서울특별시'
CITY_NAME_Eng = 'Seoul'

### 2.1 Load Predictor (Independent) Variables

In [ ]:
predictor_raw = pd.read_excel("data/cities_predictor_variables.xlsx", sheet_name=CITY_NAME_Kor)
col_name = predictor_raw.iloc[2].values
predictor_raw1 = predictor_raw.iloc[3:, :29] # header 및 일단위자료 삭제
predictor_raw1.columns = col_name[:29]
predictor_raw1_len = predictor_raw1['연도'].isna().argmax()
predictor_raw1 = predictor_raw1[:predictor_raw1_len]
predictor_raw1.index = pd.to_datetime(predictor_raw1['연도'][:predictor_raw1_len].astype(str) + '-' + predictor_raw1['월'][:predictor_raw1_len].astype(str))
predictor_raw1.rename_axis(columns='', inplace=True)
predictor_raw1.drop(columns=['연도', '월'], inplace=True)
predictor_raw1.index.name = "date"
predictor_raw1 = predictor_raw1.astype(float)
predictor_raw1.head()

In [ ]:
predictor_raw1.columns = ['Total_Population', 'Households', 'Population_per_Households', 'Male_Population', 'Female_Population', 'Male_Female_Ratio', 'Population_aging_Ratio', 
                'Power_usage', 'Num_of_Business', 'Business_above_100', 'complex_area', 'annual_household_income',
                'High_School_Graduate_num', 'High_School_Graduate_ratio', 'personal_expense', 'benefits_vs_personal_expense', 'employment_ratio', 'employment_insurance_ratio', 'vulnerable_class', 'vulnerable_class_ratio',
                'Temp', 'Rainfall', 'Humidity', 'Solar_radiation', 'Ground_Temp', 'Wind', 'Pressure']
predictor_raw1.head()

In [ ]:
predictor_f = predictor_raw1.loc["2017-01-01":"2021-12-01"]
predictor_f.head()

### 2.2 Load Response (Dependent) Variable: 용수공급량 ("월별 공급량/월별 일수"를 통해 m3/day으로 평균하여 표현)

In [ ]:
response_raw = pd.read_excel("data/water_supply(17-21).xlsx", header=1)
response_raw['date'] = response_raw["지자체명"].values
response_raw = response_raw.iloc[1:, 1:]
response_raw[f'water_supply({CITY_NAME_Eng})'] = response_raw[CITY_NAME_Kor].values
response_raw = response_raw[["date", f'water_supply({CITY_NAME_Eng})']]
response_raw.set_index("date", inplace=True)
response_raw = response_raw.astype(float)
response_raw.head()

In [ ]:
response_f = response_raw.loc["2017-01":"2021-12"]
response_f.head()

In [ ]:
dataset = predictor_f
dataset[f"water_supply({CITY_NAME_Eng})"] = response_f[f"water_supply({CITY_NAME_Eng})"].values
dataset.head()

## 3. Check Data Availability

In [ ]:
fig, ax = plt.subplots(figsize=(20,5))
sns.heatmap(dataset.isnull(), cbar=False)
sns.set(font_scale = 2.0)
plt.show()

## 4. Plot All Variables

In [ ]:
rcParams['figure.figsize'] = 15, 30
fig1 = plt.figure()
values = dataset.values
sns.set(font_scale = 0.7)
groups = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27]
i = 1
# plot each column
for group in groups:
    plt.subplot(len(groups), 1, i)
    if group == 0:
        plt.title(f"Monthly Water Supply (2017~2021) and Related Predictor Variables ({CITY_NAME_Eng})", fontsize=15)
    plt.plot(dataset.index,  values[:, group])
    plt.title(dataset.columns[group], y=0.8, loc='left')
    plt.grid(True)
    plt.xlim([dataset.index[0], dataset.index[-1]])
    i += 1
filename = CITY_NAME_Eng+"_all_variables_plot.png"
plt.savefig(filename, bbox_inches='tight')
plt.plot()  

## 5. Correlation (Categories: 1. Seasonality, 2. Locality, 3. Sociality)

### 5.1  Sesonality Features
 #### (1) Temp, (2) Rainfall, (3) Humidity, (4) Solar_radiation, (5) Ground_Temp, (6) Wind, (7) Pressure

In [ ]:
cols_seasonality =['Temp', 'Rainfall', 'Humidity', 'Solar_radiation', 'Ground_Temp', 'Wind', 'Pressure', f"water_supply({CITY_NAME_Eng})"]
plt.figure(figsize=(8, 6))
sns.set(font_scale = 1.0)
dataset[cols_seasonality].corr()
heatmap = sns.heatmap(dataset[cols_seasonality].corr(), cmap="RdYlBu_r", annot=True, annot_kws={"fontsize":12})
heatmap.set_title(f'Correlation Heatmap (Seasonality Predictor Variables vs Water Demand(Total) ({CITY_NAME_Eng}))', fontsize=15, pad=12)
filename = CITY_NAME_Eng+"_seasonality_predictor_vs_water_demand_total.png"
plt.savefig(filename, bbox_inches='tight')
plt.show()

In [ ]:
g = sns.PairGrid(dataset[cols_seasonality])
g.fig.subplots_adjust(top=0.96)
g.fig.suptitle(f'Pair Grid (Seasonality Predictor Variables vs Water Demand(Total) ({CITY_NAME_Eng}))', fontsize=20)
g.map_upper(sns.scatterplot)
g.map_lower(sns.kdeplot)
g.map_diag(sns.kdeplot, lw=3, legend=False)
plt.savefig("seasonality_predictor_vs_water_demand_total_pair_grid.png", bbox_inches='tight')

### 5.2  Locality Features 
#### 'Total_Population', 'Households', 'Population_per_Households', 'Male_Population', 'Female_Population', 'Male_Female_Ratio',                                     'Population_aging_Ratio', 'Power_usage', 'Num_of_Business', 'Business_above_100', 'complex_area'

In [ ]:
cols_locality =['Total_Population', 'Households', 'Population_per_Households', 'Male_Population', 'Female_Population', 'Male_Female_Ratio', 
                'Population_aging_Ratio', 'Power_usage', 'Num_of_Business', 'Business_above_100', 'complex_area', f"water_supply({CITY_NAME_Eng})"]
plt.figure(figsize=(8, 6))
sns.set(font_scale = 1.0)
dataset[cols_locality].corr()
heatmap = sns.heatmap(dataset[cols_locality].corr(), cmap="RdYlBu_r", annot=True, annot_kws={"fontsize":8})
heatmap.set_title(f'Correlation Heatmap (Locality Predictor Variables vs Water Demand(Total) ({CITY_NAME_Eng}))', fontsize=15, pad=12)
filename = CITY_NAME_Eng+"_locality_predictor_vs_water_demand_total.png"
plt.savefig(filename, bbox_inches='tight')
plt.show()

In [ ]:
g = sns.PairGrid(dataset[cols_locality])
g.fig.subplots_adjust(top=0.96)
g.fig.suptitle(f'Pair Grid (Locality Predictor Variables vs Water Demand(Total) ({CITY_NAME_Eng}))', fontsize=20)
g.map_upper(sns.scatterplot)
g.map_lower(sns.kdeplot)
g.map_diag(sns.kdeplot, lw=3, legend=False)
plt.savefig("locality_predictor_vs_water_demand_total_pair_grid.png", bbox_inches='tight')

### 5.3  Sociality Features 
#### 'annual_household_income', 'High_School_Graduate_num', 'High_School_Graduate_ratio', 'personal_expense', 'benefits_vs_personal_expense', 'employment_ratio', 'employment_insurance_ratio', 'vulnerable_class', 'vulnerable_class_ratio'

In [ ]:
cols_sociality =['annual_household_income', 'High_School_Graduate_num', 'High_School_Graduate_ratio', 'personal_expense', 'benefits_vs_personal_expense', 
                 'employment_ratio', 'employment_insurance_ratio', 'vulnerable_class', 'vulnerable_class_ratio', f"water_supply({CITY_NAME_Eng})"]
plt.figure(figsize=(8, 6))
sns.set(font_scale = 1.0)
dataset[cols_sociality].corr()
heatmap = sns.heatmap(dataset[cols_sociality].corr(), cmap="RdYlBu_r", annot=True, annot_kws={"fontsize":10})
heatmap.set_title(f'Correlation Heatmap (Sociality Predictor Variables vs Water Demand(Total) ({CITY_NAME_Eng}))', fontsize=15, pad=12)
filename = CITY_NAME_Eng+"_sociality_predictor_vs_water_demand_total.png"
plt.savefig(filename, bbox_inches='tight')
plt.show()

In [ ]:
g = sns.PairGrid(dataset[cols_sociality])
g.fig.subplots_adjust(top=0.96)
g.fig.suptitle(f'Pair Grid (Sociality Predictor Variables vs Water Demand(Total) ({CITY_NAME_Eng}))', fontsize=20)
g.map_upper(sns.scatterplot)
g.map_lower(sns.kdeplot)
g.map_diag(sns.kdeplot, lw=3, legend=False)
plt.savefig("sociality_predictor_vs_water_demand_total_pair_grid.png", bbox_inches='tight')

In [ ]:
data_corr = pd.concat([dataset[cols_seasonality].corr()[-1:].T[:-1], dataset[cols_locality].corr()[-1:].T[:-1], dataset[cols_sociality].corr()[-1:].T[:-1]])
data_corr.columns = [f'{CITY_NAME_Eng}_Correlation']
data_corr.sort_values(by=f'{CITY_NAME_Eng}_Correlation', ascending=True, inplace=True)
fig, ax = plt.subplots(figsize=(5,9))
ax.barh(data_corr.index, data_corr[f'{CITY_NAME_Eng}_Correlation'].values)
for i, v in enumerate(data_corr[f'{CITY_NAME_Eng}_Correlation']):
    ax.text(v, i-0.25, str(v)[0:4], fontsize=10)
    
filename = CITY_NAME_Eng+"_Correlation Ranking (Predictor Variables vs Water Demand(Total)).png"
plt.title(f'{filename.split(".")[0]}', fontsize=13)
plt.savefig(filename, bbox_inches='tight')
plt.show()    
data_corr.to_csv(f'{filename.split(".")[0]}.csv')

## 6. Data Decomposition

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
result = seasonal_decompose(dataset[f"water_supply({CITY_NAME_Eng})"], model='additive') #multiplicative

rcParams['figure.figsize'] = 18, 8
result.plot().savefig(CITY_NAME_Eng+"_data_decomposition.png")

In [ ]:
dataset["trend"] = result.trend.values
dataset["seasonal"] = result.seasonal.values
dataset["resid"] = result.resid.values
dataset.to_csv(f'{CITY_NAME_Eng}_datasets.csv')

In [ ]:
cols_seasonality =['Temp', 'Rainfall', 'Humidity', 'Solar_radiation', 'Ground_Temp', 'Wind', 'Pressure', 'seasonal']
plt.figure(figsize=(8, 6))
sns.set(font_scale = 1.0)
dataset[cols_seasonality].corr()
heatmap = sns.heatmap(dataset[cols_seasonality].corr(), cmap="RdYlBu_r", annot=True, annot_kws={"fontsize":12})
heatmap.set_title(f'Correlation Heatmap (Seasonality Predictor Variables vs Water Demand (Sesonality) ({CITY_NAME_Eng}))', fontdict={'fontsize':15}, pad=12)
filename = CITY_NAME_Eng+"_seasonality_predictor_vs_water_demand (seasonality).png"
plt.savefig(filename, bbox_inches='tight')
plt.show()

In [ ]:
g = sns.PairGrid(dataset[cols_seasonality])
g.fig.subplots_adjust(top=0.96)
g.fig.suptitle(f'Pair Grid (Seasonality Predictor Variables vs Water Demand (Sesonality) ({CITY_NAME_Eng}))', fontsize=20)
g.map_upper(sns.scatterplot)
g.map_lower(sns.kdeplot)
g.map_diag(sns.kdeplot, lw=3, legend=False)
plt.savefig(CITY_NAME_Eng+"_seasonality_predictor_vs_water_demand (seasonality)_pair_grid.png", bbox_inches='tight')

In [ ]:
cols_locality =['Total_Population', 'Households', 'Population_per_Households', 'Male_Population', 'Female_Population', 'Male_Female_Ratio', 
                'Population_aging_Ratio', 'Power_usage', 'Num_of_Business', 'Business_above_100', 'complex_area', 'trend']
plt.figure(figsize=(8, 6))
sns.set(font_scale = 1.0)
example1 = dataset[cols_locality].loc["2017-07-01":"2021-06-01"]
heatmap = sns.heatmap(example1.corr(), cmap="RdYlBu_r", annot=True, annot_kws={"fontsize":8})
heatmap.set_title(f'Correlation Heatmap (Locality Predictor Variables vs Water Demand (Trend) ({CITY_NAME_Eng}))', fontdict={'fontsize':15}, pad=12)
filename = CITY_NAME_Eng+"_locality_predictor_vs_water_demand (trend).png"
plt.savefig(filename, bbox_inches='tight')
plt.show()

In [ ]:
g = sns.PairGrid(dataset[cols_locality])
g.fig.subplots_adjust(top=0.96)
g.fig.suptitle(f'Pair Grid (Locality Predictor Variables vs Water Demand (Trend) ({CITY_NAME_Eng}))', fontsize=20)
g.map_upper(sns.scatterplot)
g.map_lower(sns.kdeplot)
g.map_diag(sns.kdeplot, lw=3, legend=False)
plt.savefig(CITY_NAME_Eng+"_locality_predictor_vs_water_demand (trend)_pair_grid.png", bbox_inches='tight')

In [ ]:
cols_locality =['Total_Population', 'Households', 'Population_per_Households', 'Male_Population', 'Female_Population', 'Male_Female_Ratio', 
                'Population_aging_Ratio', 'Power_usage', 'Num_of_Business', 'Business_above_100', 'complex_area', 'seasonal']
plt.figure(figsize=(8, 6))
sns.set(font_scale = 1.0)
example2 = dataset[cols_locality].loc["2017-07-01":"2021-06-01"]
heatmap = sns.heatmap(example2.corr(), cmap="RdYlBu_r", annot=True, annot_kws={"fontsize":8})
heatmap.set_title(f'Correlation Heatmap (Locality Predictor Variables vs Water Demand (Seasonality) ({CITY_NAME_Eng}))', fontdict={'fontsize':15}, pad=12)
filename = CITY_NAME_Eng+"_locality_predictor_vs_water_demand (seasonality).png"
plt.savefig(filename, bbox_inches='tight')
plt.show()

In [ ]:
g = sns.PairGrid(dataset[cols_locality])
g.fig.subplots_adjust(top=0.96)
g.fig.suptitle(f'Pair Grid (Locality Predictor Variables vs Water Demand (Seasonality) ({CITY_NAME_Eng}))', fontsize=20)
g.map_upper(sns.scatterplot)
g.map_lower(sns.kdeplot)
g.map_diag(sns.kdeplot, lw=3, legend=False)
plt.savefig(CITY_NAME_Eng+"_locality_predictor_vs_water_demand (seasonality)_pair_grid.png", bbox_inches='tight')

In [ ]:
cols_sociality =['annual_household_income', 'High_School_Graduate_num', 'High_School_Graduate_ratio', 'personal_expense', 'benefits_vs_personal_expense', 
                 'employment_ratio', 'employment_insurance_ratio', 'vulnerable_class', 'vulnerable_class_ratio', 'trend']
plt.figure(figsize=(8, 6))
sns.set(font_scale = 1.0)
example3 = dataset[cols_sociality].loc["2017-07-01":"2021-06-01"]
heatmap = sns.heatmap(example3.corr(), cmap="RdYlBu_r", annot=True, annot_kws={"fontsize":10})
heatmap.set_title(f'Correlation Heatmap (Sociality Predictor Variables vs Water Demand (Trend) ({CITY_NAME_Eng}))', fontdict={'fontsize':15}, pad=12)
filename = CITY_NAME_Eng+"_sociality_predictor_vs_water_demand (sociality).png"
plt.savefig(filename, bbox_inches='tight')
plt.show()

In [ ]:
g = sns.PairGrid(dataset[cols_sociality])
g.fig.subplots_adjust(top=0.96)
g.fig.suptitle(f'Pair Grid (Sociality Predictor Variables vs Water Demand (Trend) ({CITY_NAME_Eng}))', fontsize=20)
g.map_upper(sns.scatterplot)
g.map_lower(sns.kdeplot)
g.map_diag(sns.kdeplot, lw=3, legend=False)
plt.savefig(CITY_NAME_Eng+"_sociality_predictor_vs_water_demand (sociality)_pair_grid.png", bbox_inches='tight')

In [ ]:
data_corr1 = pd.concat([dataset[cols_seasonality].corr()[-1:].T[:-1], example1.corr()[-1:].T[:-1], example3.corr()[-1:].T[:-1]])
data_corr1 = data_corr1.fillna(0)
data_corr1[f'{CITY_NAME_Eng}_Correlation_decomposition'] = data_corr1['trend'] + data_corr1['seasonal']
data_corr1 = data_corr1.drop(['trend', 'seasonal'], axis=1)
data_corr1.sort_values(by=f'{CITY_NAME_Eng}_Correlation_decomposition', ascending=True, inplace=True)
fig, ax = plt.subplots(figsize=(5,9))
ax.barh(data_corr1.index, data_corr1[f'{CITY_NAME_Eng}_Correlation_decomposition'].values)
for i, v in enumerate(data_corr1[f'{CITY_NAME_Eng}_Correlation_decomposition']):
    ax.text(v, i-0.25, str(v)[0:4], fontsize=10)
filename = CITY_NAME_Eng+"_Correlation Ranking (Predictor Variables vs Water Demand (Decomposition)).png"
plt.title(f'{filename.split(".")[0]}', fontsize=13)
plt.savefig(filename, bbox_inches='tight')
plt.show()    
data_corr1.to_csv(f'{filename.split(".")[0]}.csv')